In [11]:
'''
URJC / GIA / Aprendizaje Automático 1 / Curso 23-24
alfredo.cuesta@urjc.es
'''
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
pd.set_option("display.precision", 2)

# Ingeniería de características

Como ya sabemos, los ejemplos con los que vamos a aprender nuestras máquinas se pueden presentar en forma de tabla donde:
- cada **fila** es un **ejemplo**
- cada **columna** es un **atributo** que hemos medido o registrado de los ejemplos.

En ocasiones estos atributos "tal cual" son suficientes, pero en muchas otras no nos proporcionan la información suficiente para realizar la tarea.

En esos casos una opción es crear nuevas columnas a partir de las que tenemos con la esperanza de lograr mejores resultados.

En este cuaderno comenzaremos a utilizar Scikit-Learn.


Llamaremos **vector características** (*features*) al vector con el que describimos cada ejemplo de manera definitiva, es decir el vector que finalmente se utiliza para aprender la tarea.

El vector de características puede coincidir con los atributos, pero también puede ser menor o mayor.
- **Menor**, si hemos eliminado atributos porque no resultaban útiles.
- **Mayor**, si hemos añadido nuevas columnas calculadas a partir de las que había inicialmente.
- **Igual**, si nos quedamos exactamente con los atributos dados o si creamos exactamente tantas columnas como las que eliminamos.

## Aumento de la dimensionalidad

Aumentar la dimensionalidad significa añadir nuevas característicias calculadas a partir de las que tenemos.

A diferencia de la imputación iterativa, ahora no tenemos que calcular una función de regresión; sino que somos nosotros los que elegimos el modo en el que se construyen las características.

><u>Ejemplo</u>.&nbsp; &nbsp;
Un cierto conjunto de ejemplos tiene, entre otros, dos atributos: "Población" y "Superficie". <br>
A partir de ellos podemos construir una nueva característica "Densidad de población",
$$\small
\text{Densidad de población} = \frac{\text{Población}}{\text{Superficie}}.
$$

Estas características generadas por nosotros suelen ser buenas si tenemos experiencia o conocimientos en el problema.

Esto no siempre es así, e incluso aunque tengamos experiencia es posible que no logremos dar con la expresión matemática más apropiada para generar una nueva características suficientemente "potente".

Una solución es lanzar una batería de transformaciones sobre las características; y un modo de hacerlo "organizadamente" es creando todas las combinaciones de características de un cierto grado polinómico que nosotros elegimos.

> Por ejemplo, con 2 características $x_1,~x_2$, las combinaciones de grado 2 serían: $\{x_1^2,~ x_1x_2,~ x_2^2\}$<br>
Sin embargo, la única combinación que contiene interacciones entre características es $\{x_1x_2\}$

Para ello contamos con el método  `PolynomialFeatures` de la biblioteca `sklearn.preprocessing`.

En el siguiente código podemos ver como se utiliza.

In [16]:
# Ejemplo de como hacer aumentado de dimensionalidad mediante
# características polinómicas.

degree = 2
interaction_only = False    # Si es 'True' solo calcula las interaciones entre columnas diferentes
                            # si es 'False' calcula todas las interacciones, también una columna por sí misma

df = pd.DataFrame({'x1': [1, 2, 3], 'x2': [4, 5, 6], 'x3': [7, 8, 9]})

polyf = PolynomialFeatures(degree=degree, interaction_only=interaction_only)
polyf.set_output(transform="pandas")

polyf.fit(df)                   # No se puede hacer esta línea de comando con el test NUNCA
df_poly = polyf.transform(df)   # Adapta los datos a las nuevas características


print('Dataframe inicial:')
print(df)
print('\nDataframe aumentado:')
print(df_poly)


Dataframe inicial:
   x1  x2  x3
0   1   4   7
1   2   5   8
2   3   6   9

Dataframe aumentado:
     1   x1   x2   x3  x1^2  x1 x2  x1 x3  x2^2  x2 x3  x3^2
0  1.0  1.0  4.0  7.0   1.0    4.0    7.0  16.0   28.0  49.0
1  1.0  2.0  5.0  8.0   4.0   10.0   16.0  25.0   40.0  64.0
2  1.0  3.0  6.0  9.0   9.0   18.0   27.0  36.0   54.0  81.0


Sobre el código de arriba hay que hacer los siguientes comentarios:
- Hemos creado dos variables al inicio que son las que controlan la demostración.
  - `degree` es el máximo grado de los polinomios que resultan
  - `interaction_only` sirve para quitar las características que interactuan consigo mismas.
- `polyf` es un objeto de la clase `PolynomialFeatures`.<br>
Esto es una característica común a toda la biblioteca sklearn y muy importante. <br>
Como sabemos los objetos tienen atributos; es decir que llevan dentro información.<br>
Pero además contienen métodos. En concreto `fit` y `transform` que son, junto con `predict`, los tres métodos que necesitamos en el 100% de las ocasiones.
- Primero se utiliza `fit` para que el objeto transformador `polyf` "aprenda" lo que debe hacer sobre el dataframe que le pasamos como parámetro.<br>
Después se utiliza `transform` para llevar a cabo esa transformación.
 - <u>¿Por qué separado en dos pasos?</u> <br>
 Una vez que el objeto ha aprendido a hacer la tarea, podemos aplicar el método `transform` sobre cualquier otro dataframe, con la única restricción de que tenga las mismas columnas que aquel con el que hemos hecho `fit`. <br>
 ¡Esto ocurre precisamente con el conjunto de Test!
 - <u>¿Por qué no usar `fit_transform`, que hace los dos pasos en uno solo?</u> <br>
 Yo, particularmente, NUNCA lo uso para forzar a emplear `fit` SOLO con el conjunto de entrenamiento y `transform` con el de entrenamiento y el de test.
 - <u>¿Por qué aparece una columna de unos a la izquierda del dataframe transformado?</u><br>
 Una de las tareas más frecuentes en ML es la regresión. <br>
 Esta columna de unos facilita la tarea de calcular el  *término independiente* de un modelo<br>
 &nbsp;(lo entenderemos mejor cuando lleguemos a esa parte de la asignatura).<br>
 Se puede eliminar con la opción `include_bias=False`.

## Reducción de la dimensionalidad

Los siguientes cuadernos están integramente dedicados a estas técnicas.

## Transformaciones sin modificar la dimensionalidad

Normalmente también se considera que hacer modificaciones sobre un atributo, aunque no se cree otro nuevo, es parte de la ingeniería de características.<br>
Vamos a estudiar transformaciones lineales, es decir escalados. En concreto:
  - al intervalo unidad
  - al máximo en valor absoluto
  - Estandarización

### Escalado al intervalo unidad

Dada una columna $x$, donde $x_{\rm min}~$ y $x_{\rm max}$ son el valor mínimo y máximo alcanzados, entonces la siguiente formula escala todos los valores al intervalo $[0,1]$

$$x_{\rm esc} = \frac{x - x_{\rm min}}{x_{\rm max}-x_{\rm min}}$$

- Esta operación se debe hacer columna a columna ya que los valores máximo y mínimo de $x_i$ pueden ser diferentes a los de $x_j$.
- Como resultado tenemos todas las características a la misma escala.
- Podemos utilizar `sklearn.preprocessing.MinMaxScaler`

### Escalado al máximo en valor absoluto

Dada una columna $x$, la siguiente formula escala todos los valores de modo que:
- si todos son positivos, el valor máximo es $1$ y el mínimo es mayor que cero
- si todos son negativos, el valor mínimo es $-1$ y todos son menores que cero
- si hay positivos y negativos, todos quedan transformados dentro del intervalo $[-1,1]$.


$$x_{\rm esc} = \frac{x}{\max(|x|)}$$

- Esta operación se también se debe hacer columna a columna ya que el máximo del valor absoluto de los valores de dos columnas puede ser diferente.
- Podemos utilizar `sklearn.preprocessing.MaxAbsScaler`

### Estandarización

Dada una columna $x$, con media $\mu$ y desviación $\sigma$, entonces la siguiente fórmula *estandariza* todos los valores de dicha columna
$$x_{\rm std} = \frac{x - \mu}{\sigma}$$

- Estandarizar significa qu
e tiene media cero y desviación unidad.
- Esto **NO** significa que hayamos convertido la distribución de la columna $x$ en una normal.
$$¿ ~~ x_{\rm std} \sim \mathcal{N}(0,1) ~~ ?~~ \Large \leftarrow\text{ ¡NO!}$$
- Podemos utilizar `sklearn.preprocessing.StandardScaler`


### Ejemplo

En la siguiente celda se utilizan estas técnicas para mostrar las diferencias entre ellas.

In [13]:
# función para imprimir por pantalla
def print_info(df):
  print(df)
  print(f'Mïnimos:      ',df.min().tolist())
  print(f'Máximos:      ',df.max().tolist())
  print(f'Medias:       ',df.mean().tolist())
  print(f'Desviaciones: ',df.std().tolist())

# Dataframe de juguete
df = pd.DataFrame({
    'x1': [1, 4, 2, 5, 3],
    'x2': [10, 20, 50, 30, 40],
    'x3': [-1, -4, -2, -5, -3],
    'x4': [10, -20, -50, -30, 40]
})

print(f'---Antes de los cambios----')
print_info(df)

# Escalado al intervalo unidad
scalerUnit = MinMaxScaler().set_output(transform="pandas")
scalerUnit.fit(df)  # Almacena el miìimo y máximo de cada columna
scaleUnit_df = scalerUnit.transform(df)

print(f'\n---Tras el escalado a [0,1] de cada columna----')
print_info(scaleUnit_df)

# Escalado al maximo de los valores absolutos
scalerMaxabs = MaxAbsScaler().set_output(transform="pandas")
scalerMaxabs.fit(df)  # Almacena el valor más alto de cada colunna
scalerMaxabs_df = scalerMaxabs.transform(df)

print(f'\n---Tras el escalado al maximo de los valores absolutos de cada columna----')
print_info(scalerMaxabs_df)

#  Estandarización
scalerStd = StandardScaler().set_output(transform="pandas")
scalerStd.fit(df)   # Estandariza los datos para que la media sea 0 y la desviación típica sea 1 (o valores aproximados)
scaleStd_df = scalerStd.transform(df)

print(f'\n---Tras la estandarización de cada columna----')
print_info(scaleStd_df)

---Antes de los cambios----
   x1  x2  x3  x4
0   1  10  -1  10
1   4  20  -4 -20
2   2  50  -2 -50
3   5  30  -5 -30
4   3  40  -3  40
Mïnimos:       [1, 10, -5, -50]
Máximos:       [5, 50, -1, 40]
Medias:        [3.0, 30.0, -3.0, -10.0]
Desviaciones:  [1.5811388300841898, 15.811388300841896, 1.5811388300841898, 35.35533905932738]

---Tras el escalado a [0,1] de cada columna----
     x1    x2    x3    x4
0  0.00  0.00  1.00  0.67
1  0.75  0.25  0.25  0.33
2  0.25  1.00  0.75  0.00
3  1.00  0.50  0.00  0.22
4  0.50  0.75  0.50  1.00
Mïnimos:       [0.0, 0.0, 0.0, 0.0]
Máximos:       [1.0, 1.0, 1.0, 1.0]
Medias:        [0.5, 0.5, 0.5, 0.4444444444444445]
Desviaciones:  [0.39528470752104744, 0.39528470752104744, 0.39528470752104744, 0.3928371006591931]

---Tras el escalado al maximo de los valores absolutos de cada columna----
    x1   x2   x3   x4
0  0.2  0.2 -0.2  0.2
1  0.8  0.4 -0.8 -0.4
2  0.4  1.0 -0.4 -1.0
3  1.0  0.6 -1.0 -0.6
4  0.6  0.8 -0.6  0.8
Mïnimos:       [0.2, 0.2, -1.0,

# Ejercicios

Crea un dataframe de juguete o utiliza uno que ya tengas con el único requisito de que tenga más de 5 atributos.

A continuación añádele dimensiones polinómicas de grado 2 pero SOLO de **tres** columnas que elijas, manteniendo todas las demás.


In [1]:
import pandas as pd

# Crear un diccionario con los datos
data = {
    'Nombre': ['Ana', 'Luis', 'Carlos', 'Sofía', 'Juan'],
    'Edad': [25, 31, 35, 19, 45],
    'Ciudad': ['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Bilbao'],
    'Profesión': ['Ingeniera', 'Médico', 'Profesor', 'Estudiante', 'Abogado'],
    'Salario': [2000, 3500, 2500, 0, 4000],
    'Estado civil': ['Soltera', 'Casado', 'Divorciado', 'Soltera', 'Viudo']
}

# Crear el dataframe
df = pd.DataFrame(data)

df

,Nombre,Edad,Ciudad,Profesión,Salario,Estado civil
0,Ana,25,Madrid,Ingeniera,2000,Soltera
1,Luis,31,Barcelona,Médico,3500,Casado
2,Carlos,35,Valencia,Profesor,2500,Divorciado
3,Sofía,19,Sevilla,Estudiante,0,Soltera
4,Juan,45,Bilbao,Abogado,4000,Viudo


In [2]:
# Convertir las columnas a tipo categórico
df['Ciudad'] = df['Ciudad'].astype('category')
df['Profesión'] = df['Profesión'].astype('category')
df['Estado civil'] = df['Estado civil'].astype('category')

# Codificar las columnas
codes1 = df['Ciudad'].cat.codes
code_to_categ1 = dict(zip(codes1,df['Ciudad'])) 

codes2 = df['Profesión'].cat.codes
code_to_categ2 = dict(zip(codes2,df['Profesión'])) 

codes3 = df['Estado civil'].cat.codes
code_to_categ3 = dict(zip(codes3,df['Estado civil']))

# Reemplazar las columnas originales con las codificadas
df['Ciudad'] = codes1
df['Profesión'] = codes2
df['Estado civil'] = codes3

df

,Nombre,Edad,Ciudad,Profesión,Salario,Estado civil
0,Ana,25,2,2,2000,2
1,Luis,31,0,3,3500,0
2,Carlos,35,4,4,2500,1
3,Sofía,19,3,1,0,2
4,Juan,45,1,0,4000,3


In [6]:
from sklearn.preprocessing import PolynomialFeatures

# Definir los parámetros de PolynomialFeatures
degree = 2
interaction_only = True

# Crear el objeto PolynomialFeatures
polyf = PolynomialFeatures(degree=degree, interaction_only=interaction_only)

# Supongamos que tienes un dataframe df y no quieres modificar el atributo 'Nombre'
atributo_excluido = ['Nombre', 'Ciudad', 'Profesión']

# Realiza la operación en todos los atributos excepto 'Nombre'
df_modificado = df.drop(atributo_excluido, axis=1)
df_modificado

,Edad,Salario,Estado civil
0,25,2000,2
1,31,3500,0
2,35,2500,1
3,19,0,2
4,45,4000,3


In [7]:
# Ajustar y transformar los datos
df_poly = polyf.fit(df_modificado)
df_poly = polyf.transform(df_modificado)

print('Dataframe inicial:')
print(df_modificado)
print('\nDataframe aumentado:')
df_poly = pd.DataFrame(df_poly)
df_poly

Dataframe inicial:
   Edad  Salario  Estado civil
0    25     2000             2
1    31     3500             0
2    35     2500             1
3    19        0             2
4    45     4000             3

Dataframe aumentado:


,0,1,2,3,4,5,6
0,1.0,25.0,2000.0,2.0,50000.0,50.0,4000.0
1,1.0,31.0,3500.0,0.0,108500.0,0.0,0.0
2,1.0,35.0,2500.0,1.0,87500.0,35.0,2500.0
3,1.0,19.0,0.0,2.0,0.0,38.0,0.0
4,1.0,45.0,4000.0,3.0,180000.0,135.0,12000.0


In [8]:
# Ahora, puedes unir el atributo 'Nombre' que excluiste con el dataframe modificado
df_final = pd.concat([df[atributo_excluido], df_poly], axis=1)
df_final

,Nombre,Ciudad,Profesión,0,1,2,3,4,5,6
0,Ana,2,2,1.0,25.0,2000.0,2.0,50000.0,50.0,4000.0
1,Luis,0,3,1.0,31.0,3500.0,0.0,108500.0,0.0,0.0
2,Carlos,4,4,1.0,35.0,2500.0,1.0,87500.0,35.0,2500.0
3,Sofía,3,1,1.0,19.0,0.0,2.0,0.0,38.0,0.0
4,Juan,1,0,1.0,45.0,4000.0,3.0,180000.0,135.0,12000.0


Crea características $\log (\cdot)~$ de cada una de las características del dataframe resultante del ejercicio anterior.

> *Al hacer $\log~$ estamos convirtiendo en números negativos los valores menores que uno, y en positivos más pequeños, cercanos al orden de magnitud, los valores mayores que uno.*

¿Qué problemas puede introducir esta transformación?

In [9]:
df_final.log()

# Ejecutar este bloque puede dar error cuando no existen valores 
# negativos que convertir.

AttributeError: 'DataFrame' object has no attribute 'log'

Crea características $e^{(\cdot)}~$ de cada una de las características del dataframe resultante del ejercicio anterior.

> *Al hacer $\exp~$ estamos convirtiendo todos los valores en positivos.* <br>
*Además todos los que tengan un valor absoluto mayor que uno son agrandados, y los que tiene valor absoluto menor que uno empequeñecidos.*

Descarga un dataframe de internet o utiliza uno que ya tengas con el único requisito de que tenga más de 5 atributos.
Separalo en dos:
- trainDF
- testDF

A continuación realiza ingeniería de características sobre trainDF.<br>
Debe incluir:
- Aumento de dimensionalidad, con:
  - características creadas por ti,
  - características polinómicas de orden 3, pero SOLO de 2 características que elijas y con la opción `interaction_only = True`
- Un escalado o una estandarización del dataframe resultante.

Desupués aplica esas transformaciones de testDF.

In [12]:
# Abrir bd
df = pd.read_csv('C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/DataFrames/california_housing_Completo.csv')
N, D = df.shape

In [14]:
import random

frac_test = 0.2
ind = df.index.tolist()
random.shuffle(ind)

N_test = int(N * frac_test)

testDF = df.iloc[ind[:N_test]]
trainDF = df.iloc[ind[N_test:]]

print(f'Train size: {trainDF.shape}\nTest size: {testDF.shape}')

Train size: (16000, 9)
Test size: (4000, 9)


In [19]:
# Info DF:
print(trainDF.info())   # no hay valores categóricos

# Búsqueda de NaN:
miss_data = trainDF.isna()
miss_data.sum()         # no hay NaN

# realizar un aumento de dimensionalidad

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16000 entries, 16155 to 13561
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16000 non-null  float64
 1   latitude            16000 non-null  float64
 2   housing_median_age  16000 non-null  float64
 3   total_rooms         16000 non-null  float64
 4   total_bedrooms      16000 non-null  float64
 5   population          16000 non-null  float64
 6   households          16000 non-null  float64
 7   median_income       16000 non-null  float64
 8   median_house_value  16000 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB
None


longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64